<a href="https://colab.research.google.com/github/MashaYakus/AI/blob/main/%D0%9A%D0%9F2(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy as np
from random import randint
import random
from scipy.sparse import rand

# Решение систем линейных алгебраических уравнений с несимметричными
#разреженными матрицами большой размерности. Метод бисопряженных
#градиентов.

##Типы данных

###Разреженная матрица

In [ ]:
class SparseMatrix:
    def __init__(self,
              data_dict: Dict[int, float],
              number_rows: int,
              number_columns: int):
      self.data = data_dict
      self.n_rows = number_rows
      self.n_columns = number_columns
      self.keys = sorted(self.data.keys())

    def get(self, i: int, j: int):
      if i >= self.n_rows or j >= self.n_columns:
        raise ValueError('index out of range')
      ind = i * self.n_columns + j
      if ind in self.keys:
          return self.data[ind]
      else:
          return 0.0

    def set(self, val: float, i: int, j: int):
      if i >= self.n_rows or j >= self.n_columns:
        raise ValueError('index out of range')
      ind = i * self.n_columns + j
      if ind in self.keys:
        if val != 0.0:
          self.data[ind] = val
        else:
          del self.data[ind]
          self.keys = sorted(self.data.keys())
      elif val != 0.0:
        self.data[ind] = val
        self.keys = sorted(self.data.keys())

    def __len__(self):
      return self.n_columns

###Вектор

In [ ]:
class SparseVector:
    def __init__(self,
                data_dict: Dict[int, float],
                number_elements: int):
      self.data = data_dict
      self.n_els = number_elements
      self.keys = sorted(self.data.keys())

    def get(self, i: int) -> float:
      if i >= self.n_els:
        raise ValueError('index out of range')
      if i in self.keys:
        return self.data[i]
      else:
        return 0.0

    def set(self, val: float, i: int):
      if i >= self.n_els:
        raise ValueError('index out of range')
      if i in self.keys:
        if val != 0.0:
          self.data[i] = val
        else:
          del self.data[i]
          self.keys = sorted(self.data.keys())
      elif val != 0.0:
        self.data[i] = val
        self.keys = sorted(self.data.keys())

    def __len__(self) -> int:
      return self.n_els

##Операции

###Добавление

In [ ]:
def add(vct1: SparseVector, vct2: SparseVector):
  lst1 = list(vct1.keys)
  lst2 = list(vct2.keys)
  lst1.extend(lst2)
  all_non_zero = list(set(lst1))
  res_vct = {}
  for ind in all_non_zero:
    res_vct[ind] = vct1[ind] + vct2[ind]
  return SparseVector(res_vct, len(vct1))

### Умножение

In [ ]:
def multi(object1: Union[float, SparseMatrix, SparseVector],
  object2: SparseVector,
  scalar: bool = False):
  inst1 = isinstance(object1, SparseVector)
  inst2 = isinstance(object2, SparseVector)
  inst3 = isinstance(object1, SparseMatrix)
  inst4 = isinstance(object1, float)
  if inst1 and inst2:
    return _vct_by_vct(object1, object2, scalar)
  elif inst3 and inst2:
    return _matrix_by_vct(object1, object2)
  elif inst4 and inst2:
    return _float_by_vct(object1, object2)

def _float_by_vct(num: float, vct: SparseVector):
  new_vct = {}
  if num == 0.0:
    return SparseVector({}, len(vct))
  else:
    for key in vct.keys:
      new_vct[key] = num * vct[key]
    return SparseVector(new_vct, len(vct))

def _vct_by_vct(vct1: SparseVector,
                vct2: SparseVector, scalar: bool):
  vct1_indices = list(vct1.keys)
  vct2_indices = list(vct2.keys)
  res_vct = {}
  res_val = 0.0
  flag = len(vct1_indices) < len(vct2_indices)
  iter_lst = vct1_indices if flag else vct2_indices
  for ind in iter_lst:
    mult = vct1.get(ind) * vct2.get(ind)
    if mult != 0.0:
      res_vct[ind] = mult
      res_val += mult
  if not scalar:
    return SparseVector(res_vct, len(vct1))
  else:
    return res_val

def _matrix_by_vct(matrix: SparseMatrix, vector: SparseVector):
  res_dct = {}
  row_ind = 0
  start_ind = row_ind * matrix.n_columns
  end_ind = (row_ind + 1) * matrix.n_columns - 1
  sum = 0.0
  count = 0
  for ind in matrix.keys:
    if ind >= start_ind and ind <= end_ind:
      sum += matrix[ind] * vector[ind - row_ind * matrix.n_columns]
    else:
      if sum != 0.0:
        count += 1
        res_dct[row_ind] = sum
        sum = 0.0
      row_ind += 1
      start_ind = row_ind * matrix.n_columns
      end_ind = (row_ind + 1) * matrix.n_columns - 1
      sum += matrix[ind] * vector[ind - row_ind * matrix.n_columns]
  if sum != 0.0:
    count += 1
    res_dct[row_ind] = sum
    sum = 0.0
  return SparseVector(res_dct, matrix.n_rows)


### Вычитание

In [ ]:
def sub(vct1: SparseVector, vct2: SparseVector):
  lst1 = list(vct1.keys)
  lst2 = list(vct2.keys)
  lst1.extend(lst2)
  all_non_zero = list(set(lst1))
  res_vct = {}
  for ind in all_non_zero:
    res_vct[ind] = vct1[ind] - vct2[ind]
  return SparseVector(res_vct, len(vct1))

### Нормировка

In [ ]:
def max_norm_matrix(matrix: SparseMatrix):
  row_ind = 0
  start_ind = row_ind * matrix.n_columns
  end_ind = (row_ind + 1) * matrix.n_columns - 1
  sum = 0.0
  max_val = -1.0
  for ind in matrix.keys:
    if ind >= start_ind and ind <= end_ind:
      sum += abs(matrix[ind])
    else:
      if sum != 0.0:
        if sum > max_val:
          max_val = sum
        sum = 0.0
      row_ind += 1
      start_ind = row_ind * matrix.n_columns
      end_ind = (row_ind + 1) * matrix.n_columns - 1
      sum += abs(matrix[ind])
  if sum != 0.0:
    if sum > max_val:
      max_val = sum
    sum = 0.0
  return max_val

def max_norm_vct(vector: SparseVector):
  maxv = -1.0
  for key in vector.keys:
    if abs(vector[key]) > maxv:
      maxv = abs(vector[key])
  return maxv

def norm_vct_2(vector: SparseVector):
  sum = 0.0
  #print(type(vector))
  #print(vector.keys)
  for key in vector.keys:
    sum += vector.get(key) ** 2
  return sum ** 0.5

### Транспонирование

In [ ]:
def transpose(matrix: SparseMatrix):
  new_dct = {}
  for key in matrix.keys:
    row_num, col_num = key // matrix.n_columns, key % matrix.n_columns
    new_dct[col_num * matrix.n_rows + row_num] = matrix.data[key]
  return SparseMatrix(new_dct, matrix.n_columns, matrix.n_rows)


## Алгоритм

In [ ]:
class BICG:
  def __init__(self, matrix: SparseMatrix,
              bias: SparseVector,
              iters: int,
              eps: float):
    self.matrix = matrix
    self.bias = bias
    self.iters = iters
    self.eps = eps
    self.x = SparseVector({}, len(matrix))
    self.prev_x = SparseVector({}, len(matrix))
    self.r = bias
    self.prev_r = bias
    self.p = bias
    self.prev_p = bias
    self.z = bias
    self.prev_z = bias
    self.s = bias
    self.prev_s = bias

  def bicg_iter(self) -> None:
    alpha1 = multi(self.prev_p, self.prev_r, scalar=True)
    alpha2 = multi(self.matrix, self.prev_z)
    alpha3 = multi(self.prev_s, alpha2, scalar=True)
    self.alpha = alpha1 / alpha3

    x1 = multi(self.alpha, self.prev_z)
    self.x = add(self.prev_x, x1)

    r1 = multi(self.matrix, self.prev_z)
    r2 = multi(self.alpha, r1)
    self.r = sub(self.prev_r, r2)

    p1 = multi(transpose(self.matrix), self.prev_s)
    p2 = multi(self.alpha, p1)
    self.p = sub(self.prev_p, p2)

    beta1 = multi(self.p, self.r, scalar=True)
    beta2 = multi(self.prev_p, self.prev_r, scalar=True)
    self.beta = beta1 / beta2

    z1 = multi(self.beta, self.prev_z)
    self.z = add(self.r, z1)

    s1 = multi(self.beta, self.prev_s)
    self.s = add(self.p, s1)

    self.prev_x = self.x
    self.prev_r = self.r
    self.prev_p = self.p
    self.prev_z = self.z
    self.prev_s = self.s

  def solve(self):
    start = time.time() * 1000000
    for _ in range(1, self.iters + 1):
      if norm_vct_2(self.r) < self.eps:
        break
      self.bicg_iter()
    end = time.time() * 1000000
    self.time = end - start

  def print_result(self):
    res_x = []
    res_mult = []
    res_bias = []
    for key in self.x.keys:
      res_x.append(self.x.data[key])
    for key in self.bias.keys:
      res_bias.append(self.bias.data[key])

    mult = multi(self.matrix, self.x)
    for key in mult.keys:
      res_mult.append(mult.data[key])
    print('bicg')
    print(f'result x vector: {["{:.3f}".format(i) for i in res_x]}')
    approx_vct = ["{:.3f}".format(i) for i in res_mult]
    print(f'approximated bias vector {approx_vct}')
    print(f'real bias vector {["{:.3f}".format(i) for i in res_bias]}')
    print(f'time {self.time} microseconds')
    res_vct = ["{:.3f}".format(i) for i in res_x]
    self.result_str = f'result x vector: {res_vct}\n'
    approx_bias = ["{:.3f}".format(i) for i in res_mult]
    self.result_str += f'approximated bias vector {approx_bias}\n'
    real_bias = ["{:.3f}".format(i) for i in res_bias]
    self.result_str += f'real bias vector {real_bias}\n'
    self.result_str += f'{self.time} microseconds'


# Примеры


## Матрица 20x20

In [ ]:
shape = 20
matrix_dict = {}
density = 0.1
for i in range(shape):
  for j in range(shape):
    if random.random() < density:
      matrix_dict.update({i*shape + j: randint(1,100)})
vector_b = {}
density = 0.1
for i in range(shape):
    vector_b.update({i: randint(1,100)})

In [ ]:
matrix = SparseMatrix(matrix_dict, shape, shape)
bias = SparseVector(vector_b, shape)
bicg = BICG(matrix, bias, 100, 1)
bicg.solve()
bicg.print_result()

TypeError: ignored